## Comparison of different  max time thresholds
We evaluate the impact of reducing the maximum time consumption while solving the ILP from 1h to 20 min

In [1]:
# import
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import json
import os
import pathlib

#set path to folder
folder = "../../../eval-results/_server-results/test14/results/"
net_analysis = "../../../eval-results/_server-results/net-BP-birth/net.eval" 



In [2]:
#fetch data 
#extend with net information stored in net_analysis
df_nets = pd.read_csv(net_analysis ,encoding="ISO-8859-1", skipinitialspace=True)
#df_nets['Name'] = df_nets.apply(lambda row: pnml_remover(row), axis=1)
df_nets.set_index('Name')
    
def net1adder(row):
    #get net names
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].split('-')[0].replace(".pnml","")
    else:
        return 0
def net2adder(row):        
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].split('-')[1].split('.')[0].replace(".pnml","")
    else: 
        return 0
    
def pnml_remover(row):
    #get net names
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].replace(".pnml","").replace(".rdf","")
    else:
        return row['Name']

def net1NonTaus(row):
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return df_nets.at[row['net1'],'nNonSilentTransitions'] 
    else:
        return 0
def net2NonTaus(row):
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return df_nets.at[row['net2'],'nNonSilentTransitions'] 
    else:
        return 0
def sumNonTaus(row):    
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['net1NonTaus'] + row['net2NonTaus']
    else:
        return 0

def fetch_data(folder, net_analysis, dataset, resultsFile):
    dir_list = next(os.walk(folder))[1]
    dfs = []
    #merge all 
    for subfolder in dir_list:
        #print(subfolder)
        evalFile = folder +"/" + subfolder +"/" + resultsFile
        confFile = folder +"/" + subfolder +"/config.log"
        if os.path.exists(evalFile):
            df = pd.read_csv(evalFile ,encoding="ISO-8859-1", skipinitialspace=True)
            with open(confFile) as json_file:
                conf = json.load(json_file)
                #add config information to dataframe 
                df['matcher'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - sim-weight=" + str(conf['matcher']['sim-weight']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh'] ) + " - Node Limit: " +str(conf['matcher']['ilp-node-limit']) + "- Time Limit : " +str(conf['matcher']['ilp-time-limit'])
                df['matcher_wo_weight'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh']) + " - Node Limit: " +str(conf['matcher']['ilp-node-limit']) + "- Time Limit : " +str(conf['matcher']['ilp-time-limit'])
                df['matcher_wo_limit'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - sim-weight=" + str(conf['matcher']['sim-weight']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh'] )
                df['complex-matches'] = conf['matcher']['complex matches']
                df['profile'] = conf['matcher']['profile']
                df['ilp'] =  conf['matcher']['ilp']
                df['word-sim'] =  conf['matcher']['word-sim']
                df['sim-weight'] = conf['matcher']['sim-weight']
                df['matcher-postprocessing-threshold'] = conf['matcher']['postprocessing-thresh']
                df['eval-postprocessing-threshold'] = conf['evaluation']['postprocessing-thresh']
                df['ILP-time-limit'] = conf['matcher']['ilp-time-limit']
                df['ILP-node-limit'] = conf['matcher']['ilp-node-limit']
            
            #df.set_index(['Name','matcher'])
            dfs.append(df)

    df_combined = pd.concat(dfs)

    #convert time
    df_combined['OVERALL TIME'] = df_combined['OVERALL TIME'].map(lambda x: x / 1000000000.)
    df_combined['BP TIME'] = df_combined['BP TIME'].map(lambda x: x / 1000000000.)
    df_combined['LABEL-SIM TIME'] = df_combined['LABEL-SIM TIME'].map(lambda x: x / 1000000000.)
    df_combined['LP TIME'] = df_combined['LP TIME'].map(lambda x: x / 1000000000.)
    df_combined['Dataset'] = dataset
    

    
    
    df_combined['net1'] = df_combined.apply(lambda row: net1adder(row), axis=1)
    df_combined['net2'] = df_combined.apply(lambda row: net2adder(row), axis=1)
    #df_combined['net1NonTaus'] = df_combined.apply(lambda row: net1NonTaus(row), axis=1)
    #df_combined['net2NonTaus'] = df_combined.apply(lambda row: net2NonTaus(row), axis=1)
    #df_combined['sumNonTaus'] = df_combined.apply(lambda row: sumNonTaus(row), axis=1)
    df_combined['Name'] = df_combined.apply(lambda row: pnml_remover(row), axis=1)
    return df_combined

FileNotFoundError: [Errno 2] No such file or directory: '../../../eval-results/_server-results/net-BP-birth/net.eval'

In [3]:
#time analysis
#load the three datasets
#time consumption wrt behavior share
folder_birth = "../../../eval-results/_server-results/test14/results-all/"
net_birth = "../../../eval-results/_server-results/net-BP-birth/net.eval" 
folder_uni = "../../../eval-results/_server-results/test19/results-all/"
net_uni = "../../../eval-results/_server-results/net-BP-uni/net.eval" 
folder_sap = "../../../eval-results/_server-results/test20/results-all/"
net_sap = "../../../eval-results/_server-results/net-BP-sap/net.eval" 
df_compined = {}
df_combined = fetch_data(folder_birth, net_birth ,"Birth","aggResults.eval")
df_combined = df_combined.append(fetch_data(folder_uni, net_uni, "Uni","aggResults.eval"))
df_combined = df_combined.append(fetch_data(folder_sap, net_sap, "Sap","aggResults.eval"))


df_ = df_combined[df_combined["Name"] != "Aggregated (MICRO)"]
df_ = df_[df_["Name"] != "Aggregated (MACRO)"]
df_["ILP-time-limit"] = ["%s s"  % x for x in df_["ILP-time-limit"]]

#export to excel
df_.to_excel("time-comparison-1h-20min.xlsx")
print(df_[df_["Dataset"] =="Uni"])
#add avg + 90 percentils
quantile = df_.groupby(['sim-weight','ILP-time-limit']).quantile([0.90])['LP TIME'].unstack()
quantile = quantile.rename(columns={0.90: "Quantile"})
quantile["Null"] = 0
quantile.reset_index(inplace = True) 
print(quantile)

# time wrt behavioral share
fig, (ax1) = plt.subplots(1,1)
fig.set_size_inches(15.5, 5.25)
colors = sns.color_palette("pastel")

#areas
#plt.fill_between(quantile[quantile["ILP-time-limit"] == "3600.0 s"]["sim-weight"],quantile[quantile["ILP-time-limit"] == "3600.0 s"]["Quantile"], alpha=0.6, color=colors[1])
#plt.fill_between(quantile[quantile["ILP-time-limit"] == "1200.0 s"]["sim-weight"],quantile[quantile["ILP-time-limit"] == "1200.0 s"]["Quantile"], alpha=0.6, color=colors[0])

#lines
#plt.plot(quantile[quantile["ILP-time-limit"] == "1200.0 s"]["sim-weight"],quantile[quantile["ILP-time-limit"] == "1200.0 s"]["Quantile"], color=colors[0])
#plt.plot(quantile[quantile["ILP-time-limit"] == "3600.0 s"]["sim-weight"],quantile[quantile["ILP-time-limit"] == "3600.0 s"]["Quantile"], color=colors[1])

sns.lineplot(x="sim-weight", y="LP TIME", hue="ILP-time-limit", data=df_, ax=ax1, ci=0);
ax1.set_ylabel("ILP Time (s)")
ax1.set_xlabel("Similarity Weight")
plt.title("ILP Time with respect to behavioral share")


NameError: name 'fetch_data' is not defined

In [4]:
# results of precision recall and fscore
# we conclude the error on the result is pretty low when setting the time limit lower
folder_birth = "../../../eval-results/_server-results/test14_2"
net_birth = "../../../eval-results/_server-results/net-BP-birth/net.eval" 
folder_uni = "../../../eval-results/_server-results/test19_3"
net_uni = "../../../eval-results/_server-results/net-BP-uni/net.eval" 
folder_sap = "../../../eval-results/_server-results/test20_3"
net_sap = "../../../eval-results/_server-results/net-BP-sap/net.eval" 

# load the three datasets
df_combined = fetch_data(folder_birth, net_birth ,"Birth","aggRetrospectiveResults.eval")
df_combined = df_combined.append(fetch_data(folder_uni, net_uni, "Uni","aggRetrospectiveResults.eval"))
df_combined = df_combined.append(fetch_data(folder_sap, net_sap, "Sap","aggRetrospectiveResults.eval"))

# Avg Precisision Recall Fscore over postprocessing threshold
df_macro_avg = df_combined[df_combined.Name == "Aggregated (MACRO)"]
df_micro_avg = df_combined[df_combined.Name == "Aggregated (MICRO)"]

def relative_diff(df):  
    df_new = pd.DataFrame(columns=['RECALL', 'PRECISION', 'FSCORE','sim-weight','eval-postprocessing-threshold','Dataset'])
    
    for dataset in df['Dataset'].unique():
        for sim in df['sim-weight'].unique():
        #for value in ppthresh
            for ppt in df['eval-postprocessing-threshold'].unique():
            #get both columns
                
                
                x_ref = df[(df['Dataset'] == dataset) & (df['sim-weight'] == sim) & (df['eval-postprocessing-threshold'] == ppt) & (df['ILP-time-limit'] == 3600)]
                x = df[(df['Dataset'] == dataset) & (df['sim-weight'] == sim) & (df['eval-postprocessing-threshold'] == ppt) & (df['ILP-time-limit'] == 1200)]            
                
                if (x_ref.empty) or (x.empty):
                    print("Combination not found")
                    print(ppt)
                    print(sim)
                    print(dataset)
                    continue;
                
                recall = abs(x_ref.iloc[0]['RECALL'] - x.iloc[0]['RECALL']) / x_ref.iloc[0]['RECALL']
                precision = abs(x_ref.iloc[0]['PRECISION'] - x.iloc[0]['PRECISION']) / x_ref.iloc[0]['PRECISION']
                fscore = abs(x_ref.iloc[0]['FSCORE'] - x.iloc[0]['FSCORE']) / x_ref.iloc[0]['FSCORE']
                df_new.loc[len(df_new)] = [recall,precision,fscore,sim,ppt,dataset]
            #create new column in new data frame
    return df_new
#calculate the difference between the maximum recall/precision/fscore and the minimum
#thats how we show that the reduction of time does not effect the results
df_macro_avg = relative_diff(df_macro_avg) #df_macro_avg.groupby(['sim-weight','eval-postprocessing-threshold'])['RECALL','PRECISION','FSCORE'].agg(np.ptp)
#df_macro_avg.reset_index(inplace=True)
df_micro_avg =  relative_diff(df_micro_avg) #df_micro_avg.groupby(['sim-weight','eval-postprocessing-threshold',])['RECALL','PRECISION','FSCORE'].agg(np.ptp)
#df_micro_avg.reset_index(inplace=True) 

        
# fix a seaborn bug that doesnt allow for categorical hue
diff_values = len(df_macro_avg.groupby(['sim-weight']))
df_macro_avg['sim-weight'] = df_macro_avg['sim-weight'].astype(str)
df_micro_avg['sim-weight'] = df_micro_avg['sim-weight'].astype(str)

#write to excel
with pd.ExcelWriter('time-comparison-error.xlsx') as writer:  
    df_macro_avg.to_excel(writer, sheet_name='Macro')
    df_micro_avg.to_excel(writer, sheet_name='Micro')
    df_combined.to_excel(writer, sheet_name='All')

#visualize it
#fig, (ax1, ax2) = plt.subplots(1,2)
#fig.set_size_inches(18.5, 5.25)
#fig.suptitle("Difference between 1 hour and 20 minutes max ILP runtime")
#sns.lineplot(x="eval-postprocessing-threshold", y="RECALL", hue="sim-weight", data=df_macro_avg, ax = ax1,  palette=sns.color_palette(None,diff_values));
#ax1.set_title("Macro")
#ax1.get_legend().remove()
#sns.lineplot(x="eval-postprocessing-threshold", y="RECALL", hue="sim-weight", data=df_micro_avg, ax = ax2,  palette = sns.color_palette(None,diff_values));
#ax2.set_title("Micro")
#ax2.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#plt.show()

#fig, (ax1, ax2) = plt.subplots(1,2)
#fig.suptitle("Difference between 1 hour and 20 minutes max ILP runtime")
#fig.set_size_inches(18.5, 5.25)
#sns.lineplot(x="eval-postprocessing-threshold", y="PRECISION", hue="sim-weight", data=df_macro_avg, ax = ax1, palette= sns.color_palette(None,diff_values));
#ax1.set_title("Macro")
#ax1.get_legend().remove()
#sns.lineplot(x="eval-postprocessing-threshold", y="PRECISION", hue="sim-weight", data=df_micro_avg, ax = ax2, palette= sns.color_palette(None,diff_values));
#ax2.set_title("Micro")
#ax2.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#plt.show()

fig, ax = plt.subplots(1,2)
fig.tight_layout()
#fig.suptitle("Avg. Relative Difference in Fscore: 1 hour compared to 20 min ILP runtime")
fig.set_size_inches(12.5, 4)
#ax[0][0].set_ylim(0.0,0.08)
sns.lineplot(x="eval-postprocessing-threshold", y="FSCORE", hue="Dataset", data=df_macro_avg, ax = ax[0], ci="sd");
ax[0].set_title("Macro")
ax[0].set_ylabel("Rel. Difference in Fscore (macro)")
ax[0].set_xlabel("Postprocessing Threshold")
ax[0].set_ylim(0.0,1.0)
#ax1.get_legend().remove()
#ax[0][1].set_ylim(0.0,0.08)
sns.lineplot(x="eval-postprocessing-threshold", y="FSCORE", hue="Dataset", data=df_micro_avg, ax = ax[1], ci="sd");
ax[1].set_title("Micro")
ax[1].set_ylabel("Rel. Difference in Fscore (micro)")
ax[1].set_xlabel("Postprocessing Threshold")
ax[1].set_ylim(0.0,1.0)
#ax2.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

fig, ax = plt.subplots(1,2)
fig.tight_layout()
#fig.suptitle("Avg. Relative Difference in Fscore: 1 hour compared to 20 min ILP runtime")
fig.set_size_inches(12.5, 4)
ax[0].set_ylim(0.0,1.0)
sns.lineplot(x="sim-weight", y="FSCORE", data=df_macro_avg, hue="Dataset", ax = ax[0], ci="sd");
ax[0].set_title("Macro")
ax[0].set_ylabel("Rel. Difference in Fscore (macro)")
ax[0].set_xlabel("Similarity Weight")
#ax1.get_legend().remove()
ax[1].set_ylim(0.0,1.0)
sns.lineplot(x="sim-weight", y="FSCORE", data=df_micro_avg, hue="Dataset", ax = ax[1], ci="sd");
ax[1].set_title("Micro")
ax[1].set_ylabel("Rel. Difference in Fscore (micro)")
ax[1].set_xlabel("Similarity Weight")
#ax2.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

NameError: name 'fetch_data' is not defined

In [5]:
#read gurobi file function
def read_gurobi(file):
    delete = True
    objective_value = -1
    lastLine = ""
    cleaned = "cleaned.log"
    infile = open(file,'r').readlines()
    with open(cleaned,'w') as outfile:
        for index, line in enumerate(infile):  
            if "Cutting plane" in line:
                delete = True
            if "Explored" in line:
                delete = True
            if "Best" in line and "objective" in line:
                objective_value = line.split()[2].replace(',','')
            if " Expl" in line and "It/Node Time" in line:
                delete = False
            line = line.replace('H','').replace('s','').replace('|.1','').replace('|.2','').replace('|','')
            if not delete:
                if len(lastLine.split())> 1 and len(line.split())> 1 and lastLine.split()[-1] != line.split()[-1]:
                    outfile.write(lastLine)
                words = line.split()
                if len(words) == 7: #Heurisitc found solution
                    words.insert(2,'0')
                    words.insert(3,'0')
                    words.insert(4,'0')
                    line = " ".join(words)
                    line+="\n"
                if len(words) >0:
                    lastLine = line
        outfile.write(lastLine)
    
    
    #handle empty dataframes, resulting from immediate solving
    if os.stat(cleaned).st_size == 0:
        gurobi_df = pd.DataFrame()
        gurobi_df = gurobi_df.append(pd.Series(), ignore_index=True)
        gurobi_df['Expl'] = 0
        gurobi_df['Unexpl'] = 0
        gurobi_df['IntInf'] = 0
        gurobi_df['Depth'] = 0
        gurobi_df['Incumbent'] = float(objective_value)
        gurobi_df['Obj'] = 0
        gurobi_df['BetBd'] = float(objective_value)
        gurobi_df['Gap'] = 0.00
        gurobi_df['Time'] = 0
        gurobi_df['It/Node'] = 0
    else:
        gurobi_df =pd.read_csv(cleaned, delim_whitespace=True, encoding="ISO-8859-1")
    gurobi_df=gurobi_df.dropna()
        #drop duplicate columns resulting from kicking Heuristic solutions
    gurobi_df.drop_duplicates(keep='last',subset=['Time'])
    
    if gurobi_df.empty:
        gurobi_df = pd.DataFrame()
        gurobi_df = gurobi_df.append(pd.Series(), ignore_index=True)
        gurobi_df['Expl'] = 0
        gurobi_df['Unexpl'] = 0
        gurobi_df['IntInf'] = 0
        gurobi_df['Depth'] = 0
        gurobi_df['Incumbent'] = float(objective_value)
        gurobi_df['Obj'] = 0
        gurobi_df['BetBd'] = float(objective_value)
        gurobi_df['Gap'] = 0.00
        gurobi_df['Time'] = 0
        gurobi_df['It/Node'] = 0
    
    #print(gurobi_df)
    #add missing values
    last_row = gurobi_df.iloc[[0]]
    for i in range(3601):
        #if there is a at least one row for that time stamp take the last row of that stamp
        #if not inser new row but with time shifted
        if (gurobi_df['Time'] == i).any():
            last_row = gurobi_df.loc[gurobi_df['Time'] == i].iloc[[0]] 
        else:
            last_row['Time'] = i
            #print(last_row)
            gurobi_df = gurobi_df.append(last_row, ignore_index = True)
    
    # only every 10 sec to reduce data
    #gurobi_df = gurobi_df.iloc[::10, :]
    
    
    #typing
    gurobi_df['Expl'] = gurobi_df['Expl'].astype(int)
    gurobi_df['Time'] = gurobi_df['Time'].astype(int)
    gurobi_df['Incumbent'] = gurobi_df['Incumbent'].astype(float)
    
    #set final found objective
    gurobi_df['final objective'] = round(float(objective_value),5)
    
    #Difference between final objective value and incumbent
    gurobi_df['Incumbent'] = gurobi_df.apply(lambda row: rounding(row), axis=1)
    gurobi_df['abs_diff'] = gurobi_df.apply(lambda row: difference(row), axis=1)
    gurobi_df['rel_diff'] = gurobi_df.apply(lambda row: rel_difference(row), axis=1)
    return gurobi_df

def rounding(row):
    return round(float(row['Incumbent']),5)
                
def rel_difference(row):
    if row['final objective'] < 0:
        return 0
    else:
        return (row['final objective'] - row['Incumbent']) / row['final objective']
    

def difference(row): 
    if row['final objective'] < 0:
        return 0
    else:
        return row['final objective'] - row['Incumbent']

In [6]:
#THIS TAKES HELL OF A TIME! CAUTION
folder_birth = "../../../eval-results/_server-results/test14/gurobi-logs/"
folder_uni = "../../../eval-results/_server-results/test19/gurobi-uni/"
folder_sap = "../../../eval-results/_server-results/test20/gurobi-sap/"
df_gurobi_combined = []

def loadFiles(folder, label):
    df_gurobi_c = []
    dir_list = next(os.walk(folder))[2]
    for index,file in  enumerate(dir_list):
        if ".log" in file:
            df_gurobi = read_gurobi(folder + file)
            df_gurobi['file'] = index
            df_gurobi['Dataset'] = label
            df_gurobi_c.append(df_gurobi)
    df_gurobi_c = pd.concat(df_gurobi_c)  
    #print(df_gurobi_c)
    return df_gurobi_c
df_gurobi_combined = loadFiles(folder_birth, "Birth")
df_gurobi_combined = df_gurobi_combined.append(loadFiles(folder_uni, "Uni"))
df_gurobi_combined = df_gurobi_combined.append(loadFiles(folder_sap, "Sap"))
#print(df_gurobi_combined)

#write to excel
df_gurobi_combined.to_excel("gurobi-time-comparison.xlsx", sheet_name='Gurobi')  

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,5))
fig.suptitle("Difference from Incumbent to found maximal objective")
fig.set_size_inches(18.5, 5.25)
ax1.set(yscale="log")
sns.lineplot(x="Time", y="rel_diff", hue="Dataset", data=df_gurobi_combined, ax = ax1);
ax1.set_title("by time")

ax2.set(yscale="log", xscale="log")
ax2.set_title("by explored nodes")
sns.lineplot(x="Expl", y="rel_diff", hue="Dataset", data=df_gurobi_combined, ax = ax2);

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,5))
fig.suptitle("Difference from Incumbent to found maximal objective")
fig.set_size_inches(18.5, 5.25)
ax1.set(yscale="log")
sns.lineplot(x="Time", y="rel_diff", data=df_gurobi_combined, ax = ax1);
ax1.set_title("by time")



StopIteration: 

In [7]:
# load dataset
df_gurobi_combined_2 = pd.read_csv('gurobi-logs.csv') 

#print(df_gurobi_combined_2[(df_gurobi_combined_2["Dataset"] == "Uni") & (df_gurobi_combined_2["Time"] ==2700) & (df_gurobi_combined_2["rel_diff"] ==0)])
#print(df_gurobi_combined_2[(df_gurobi_combined_2["Dataset"] == "Uni") & (df_gurobi_combined_2["Time"] ==2700) &((df_gurobi_combined_2["Gap"] == "0.00%") | (df_gurobi_combined_2["Gap"] == "0.0") | (df_gurobi_combined_2["Gap"] == 0 ) | (df_gurobi_combined_2["Gap"] == 0.00))])

#add avg + 90 percentils
quantile = df_gurobi_combined_2.groupby(['Dataset','Time']).quantile([0.90])['rel_diff'].unstack()
quantile = quantile.rename(columns={0.90: "Quantile"})
quantile["Null"] = 0
quantile.reset_index(inplace = True) 

mean = df_gurobi_combined_2.groupby(['Dataset','Time']).mean()
mean.reset_index(inplace = True) 
print(quantile[quantile["Time"] == 1200])

fig, (ax1) = plt.subplots(1,1, figsize=(3,1))
ax1.set_title("Average relative difference from Incumbent to found Maximal Objective after 1h ILP runtime")
fig.set_size_inches(12.5, 5.25)
#ax1.set(yscale="log")
#ax1.set(xscale="log")
plt.xlim(1,3620)
plt.ylim(0.0003,0.2)
plt.axvline(x=1200, color='k', linestyle='--')
plt.text(1200, 0.08, '20 minutes', ha='center', va='center',rotation='vertical', backgroundcolor='white')


#plot 90 percentiles
colors = sns.color_palette("pastel")
#areas
plt.fill_between(quantile[quantile["Dataset"] == "Uni"]["Time"],quantile[quantile["Dataset"] == "Uni"]["Quantile"], alpha=0.6, color=colors[1])
plt.fill_between(quantile[quantile["Dataset"] == "Birth"]["Time"],quantile[quantile["Dataset"] == "Birth"]["Quantile"], alpha=0.6, color=colors[0])
plt.fill_between(quantile[quantile["Dataset"] == "Sap"]["Time"],  quantile[quantile["Dataset"] == "Sap"]["Quantile"],   alpha=0.6, color=colors[2])
#lines
plt.plot(quantile[quantile["Dataset"] == "Uni"]["Time"],quantile[quantile["Dataset"] == "Uni"]["Quantile"], color=colors[1])
plt.plot(quantile[quantile["Dataset"] == "Birth"]["Time"],quantile[quantile["Dataset"] == "Birth"]["Quantile"], color=colors[0])
plt.plot(quantile[quantile["Dataset"] == "Sap"]["Time"],quantile[quantile["Dataset"] == "Sap"]["Quantile"], color=colors[2])

sns.lineplot(x="Time", y="rel_diff", data=mean, ax = ax1,hue="Dataset", hue_order =["Birth", "Uni", "Sap"]); #plot mean


ax1.set_ylabel("Relative Difference")
ax1.set_xlabel("Time (in seconds)")


FileNotFoundError: [Errno 2] No such file or directory: 'gurobi-logs.csv'

In [8]:
df_gurobi_combined.to_csv("gurobi-logs.csv", sep=',')

AttributeError: 'list' object has no attribute 'to_csv'